# Permutations of a Simple Circuit

This notebook walks through how to utilize the core semantics of SysML v2 to generate alternative circuits as inputs to an OpenMDAO solution of these circuits. 

## Background

The M1 user model in SysML v2 is meant to be a set of constraints and rules under which legal instances can be created. Those instances should be taken as alternative produced systems and they can be analyzed in that way.

## Libraries Load-Up

Load up PyMBE and its various libraries.

In [ ]:
from pathlib import Path
import networkx as nx
import matplotlib as plt

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
    random_generator_phase_1_multiplicities,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Load Up Model

Read the model from the local JSON file.

In [ ]:
parts_client = SysML2Client()

simple_parts_file = Path(pm.__file__).parent / "../../tests/fixtures/Circuit Builder.json"

parts_client._load_from_file(simple_parts_file)

parts_lpg = SysML2LabeledPropertyGraph()
parts_lpg.model = parts_client.model

SIMPLE_MODEL = "Model::Simple Parts Model::"

[id_to_parts_name_lookup, parts_name_to_id_lookup] = build_stable_id_lookups(parts_lpg)

parts_lpg.model.MAX_MULTIPLICITY = 10

## Explore Contents of Model with M1 in Memory

Use the M1 memory objects to see what is in the current model, starting with the main packages.

In [ ]:
parts_lpg.model.packages

In [ ]:
parts_lpg.model.ownedElement["Circuit Builder"].ownedElement

In [ ]:
circuit_def = parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"]

### Circuit and its Features

Here is the circuit and its features, both parts and used connections.

In [ ]:
circuit_def.relationships

In [ ]:
circuit_def.ownedMember

In [ ]:
id_to_parts_name_lookup[circuit_def.ownedMember["Circuit Diode"]._id]

#### Feature types for sequences

We can also inspect the types of each of the features (or classifier themselves) to see what will be placed into each step of the sequence.

In the below, what we mean to say is that for a given position, the atom in that place will also appear in the 1-tail of the given type. For example, the type that corresponds to the Power User: Part::Power In: Port in the second position, which is a Part, will have all atoms in the 1-tail of Part's sequences. 

In [ ]:
feature_templates_with_ids = [[item for item in seq] for seq in build_sequence_templates(parts_lpg)
                                    if id_to_parts_name_lookup[seq[0]].startswith("Model::Circuit Builder")]
feature_templates_with_names = [[id_to_parts_name_lookup[item] for item in seq] for seq in build_sequence_templates(parts_lpg)
                                    if id_to_parts_name_lookup[seq[0]].startswith("Model::Circuit Builder")]
feature_templates_with_names

In [ ]:
[
    [
        [id_to_parts_name_lookup[typ] for typ in get_types_for_feature(parts_lpg, parts_lpg.model.elements[item])]
    for item in seq
    ]
for seq in feature_templates_with_ids
]

In [ ]:
expr_templates_with_ids = [[item for item in seq] for seq in build_expression_sequence_templates(parts_lpg)
                                    if id_to_parts_name_lookup[seq[0]].startswith("Model::Circuit Builder")]
expr_templates_with_names = [[id_to_parts_name_lookup[item] for item in seq] for seq in build_expression_sequence_templates(parts_lpg)
                                    if id_to_parts_name_lookup[seq[0]].startswith("Model::Circuit Builder")]
expr_templates_with_names

#### Exploring Connections and Ends

An important kind of feature is the feature that is typed by a Connection, which has two ends.

In [ ]:
connection_usage = [usage for usage in parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedElement
                        if usage._metatype == "ConnectionUsage"][0]
connection_usage

In [ ]:
ptg = parts_lpg.get_projection("Part Typing")
scg = parts_lpg.get_projection("Part Definition")
ssg = parts_lpg.get_projection("Redefinition and Subsetting")
bdg = parts_lpg.get_projection(projection="Expanded Banded", packages=[parts_lpg.model.ownedElement["Occurrences"]])
bdg_all = parts_lpg.get_projection(projection="Expanded Banded")

full_multiplicities = random_generator_phase_1_multiplicities(parts_lpg, ptg, scg)

In [ ]:
{
    id_to_parts_name_lookup[k]:v
    for k, v in full_multiplicities.items()
}

In [ ]:
{
    k:v
    for k, v in full_multiplicities.items()
}

In [ ]:
[id_to_parts_name_lookup[typ_] for typ_ in get_features_typed_by_type(parts_lpg, '81b05f03-a17a-4f51-83a2-06d7f19d5c6d')]

In [ ]:
nx.is_directed_acyclic_graph(ptg)

In [ ]:
nx.is_directed_acyclic_graph(scg)

In [ ]:
nx.is_directed_acyclic_graph(ssg)

In [ ]:
nx.is_directed_acyclic_graph(bdg)

In [ ]:
len(ssg.nodes)

In [ ]:
len(ssg.edges)

In [ ]:
[id_to_parts_name_lookup[node] for listing in nx.simple_cycles(ssg) for node in listing]

In [ ]:
[
    id_to_parts_name_lookup[node]
    for node in bdg_all.nodes
    if bdg_all.out_degree(node) < 1
]

In [ ]:
[[id_to_parts_name_lookup[node] for node in listing] for listing in nx.simple_cycles(bdg_all)]

In [ ]:
item_def_id = parts_name_to_id_lookup["Model::Items::Item <<ItemDefinition>>"]
done_item_id = parts_name_to_id_lookup["Model::Items::Item::done: Item <<ItemUsage>>"]

In [ ]:
list(nx.all_simple_paths(
    bdg_all,
    source=item_def_id,
    target=done_item_id,
))

In [ ]:
[id_to_parts_name_lookup[item] for item in bdg_all.predecessors(item_def_id)]

In [ ]:
cycle_check = [node for listing in nx.simple_cycles(ssg) for node in listing]

In [ ]:
list(ssg.predecessors(cycle_check[1]))

In [ ]:
[parts_lpg.model.elements[item] for item in parts_lpg.get_projection("Generalization")]

In [ ]:
m0_interpretation = random_generator_playbook(
    lpg=parts_lpg,
    name_hints={},
    filtered_feat_packages=[parts_lpg.model.ownedElement["Circuit Builder"]],
    phase_limit=10
)

In [ ]:
parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit Connection"]

In [ ]:
parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedMember["Component"]

In [ ]:
m0_interpretation[parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit Connection"]._id]

In [ ]:
m0_interpretation[parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedMember["Circuit Resistor"]._id]

In [ ]:
m0_interpretation[parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedMember["Circuit Diode"]._id]

In [ ]:
m0_interpretation[parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedMember["Motive Force"]._id]

In [ ]:
m0_interpretation[parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["EMF"]._id]

In [ ]:
dict(nx.bfs_successors(parts_lpg.get_projection("Generalization"), '9b93e6a8-d44e-4e17-a859-9cdedf60c144'))

In [ ]:
parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedMember["Circuit Diode"]._id

In [ ]:
m0_interpretation[parts_lpg.model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedMember["Component"]._id]

In [ ]:
pprint_interpretation(m0_interpretation, parts_lpg.model)